In [2]:
#for DataFrame we use a SparkSession instead of a SparkContext
from pyspark.sql import SparkSession
# SparkSession uses the "builder" syntax
spark = SparkSession.builder.master('local[*]').appName('Spark DataFrame').getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%configure -f
{"executorMemory": "2G","driverMemory":"1G","executorCores": 1,"numExecutors": 6, 
 "conf": {"spark.dynamicAllocation.enabled": "false", 
          "spark.sql.parquet.enableVectorizedReader": "false", 
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type": "native",
          "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv"}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
161,application_1713565790653_0161,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
126,application_1713565790653_0126,pyspark,idle,Link,Link,None,
138,application_1713565790653_0138,pyspark,idle,Link,Link,None,
140,application_1713565790653_0140,pyspark,idle,Link,Link,None,
142,application_1713565790653_0142,pyspark,idle,Link,Link,None,
143,application_1713565790653_0143,pyspark,idle,Link,Link,None,
154,application_1713565790653_0154,pyspark,idle,Link,Link,None,
159,application_1713565790653_0159,pyspark,idle,Link,Link,None,
161,application_1713565790653_0161,pyspark,idle,Link,Link,None,✔


## **_Read and Combine dataframe_**

In [4]:
books = spark.read.options(inferSchema=True, multiLine=True, escape='\"', header=True).csv('s3://msbx5420-spr24/teams/Group8/Books.csv')
books.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|      ISBN|         Book-Title|         Book-Author|Year-Of-Publication|           Publisher|         Image-URL-S|         Image-URL-M|         Image-URL-L|
+----------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|0195153448|Classical Mythology|  Mark P. O. Morford|               2002|Oxford University...|http://images.ama...|http://images.ama...|http://images.ama...|
|0002005018|       Clara Callan|Richard Bruce Wright|               2001|HarperFlamingo Ca...|http://images.ama...|http://images.ama...|http://images.ama...|
+----------+-------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows

In [5]:
ratings = spark.read.options(inferSchema=True, multiLine=True, escape='\"', header=True).csv('s3://msbx5420-spr24/teams/Group8/Ratings.csv')
ratings.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+-----------+
|User-ID|      ISBN|Book-Rating|
+-------+----------+-----------+
| 276725|034545104X|          0|
| 276726|0155061224|          5|
+-------+----------+-----------+
only showing top 2 rows

In [6]:
users = spark.read.options(inferSchema = True, multiLine = True, escape = '\"', header=True).csv('s3://msbx5420-spr24/teams/Group8/Users.csv')
users.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+----+
|User-ID|            Location| Age|
+-------+--------------------+----+
|      1|  nyc, new york, usa|null|
|      2|stockton, califor...|18.0|
+-------+--------------------+----+
only showing top 2 rows

In [7]:
ratings_books = ratings.join(books, on='ISBN', how='inner')
ratings_books.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+-----------+--------------------+-------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|      ISBN|User-ID|Book-Rating|          Book-Title|  Book-Author|Year-Of-Publication|           Publisher|         Image-URL-S|         Image-URL-M|         Image-URL-L|
+----------+-------+-----------+--------------------+-------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|0001900277|  11676|          0|Glue (First Facts...|Harriet Hains|               1989|HarperCollins Pub...|http://images.ama...|http://images.ama...|http://images.ama...|
|0001900277| 181659|         10|Glue (First Facts...|Harriet Hains|               1989|HarperCollins Pub...|http://images.ama...|http://images.ama...|http://images.ama...|
+----------+-------+-----------+--------------------+-------------+-------------------+--------------------+--------------------+-----------

In [8]:
complete_data = ratings_books.join(users, on='User-ID', how='inner')
complete_data.show(2)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+-----------+--------------------+--------------------+-------------------+------------------+--------------------+--------------------+--------------------+--------------------+----+
|User-ID|      ISBN|Book-Rating|          Book-Title|         Book-Author|Year-Of-Publication|         Publisher|         Image-URL-S|         Image-URL-M|         Image-URL-L|            Location| Age|
+-------+----------+-----------+--------------------+--------------------+-------------------+------------------+--------------------+--------------------+--------------------+--------------------+----+
|  26583|0064640329|          0|Children Through ...|Barbara Kaye Gree...|               1979|Barnes &amp; Noble|http://images.ama...|http://images.ama...|http://images.ama...|dollard des ormea...|null|
| 119517|0140293248|          0|The Girls' Guide ...|        Melissa Bank|               2000|     Penguin Books|http://images.ama...|http://images.ama...|http://images.ama...|andover, min

In [9]:
column_names = complete_data.columns
print(column_names)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['User-ID', 'ISBN', 'Book-Rating', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L', 'Location', 'Age']

In [10]:
complete_data.sort("Year-Of-Publication", ascending = True).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+-----------+--------------------+------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+
|User-ID|      ISBN|Book-Rating|          Book-Title|       Book-Author|Year-Of-Publication|           Publisher|         Image-URL-S|         Image-URL-M|         Image-URL-L|            Location| Age|
+-------+----------+-----------+--------------------+------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----+
| 198711|0590404008|          0|     Hundred Dresses|     Eleanor Estes|                  0|      Scholastic Inc|http://images.ama...|http://images.ama...|http://images.ama...|little canada, mi...|62.0|
| 158295|1566199735|          5|    A Z Aromatherapy|    Patricia Davis|                  0|Dorset House Publ...|http://images.ama...|http://images.ama...|http://images.ama...|cleveland, o

In [11]:
abook = complete_data.select('User-ID', 'ISBN', 'Book-Rating', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher','Location', 'Age' )
abook.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+-----------+--------------------+-------------------+-------------------+----------------+--------------------+----+
|User-ID|      ISBN|Book-Rating|          Book-Title|        Book-Author|Year-Of-Publication|       Publisher|            Location| Age|
+-------+----------+-----------+--------------------+-------------------+-------------------+----------------+--------------------+----+
|  26583|0140507922|         10|         Bear Circus|William Pene Dubois|               1987|     Penguin USA|dollard des ormea...|null|
| 104880|0345339703|          9|The Fellowship of...|     J.R.R. TOLKIEN|               1986|         Del Rey|madison, wisconsi...|21.0|
| 104880|0345361989|          0|Guests of the Emp...|Janice Young Brooks|               1992|Ballantine Books|madison, wisconsi...|21.0|
| 275481|0345446666|          8|Death in Holy Ord...|         P.D. JAMES|               2002|         Fawcett|tucson, arizona, usa|64.0|
| 240209|0345450418|          8|   Imposs

In [12]:
abook.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- User-ID: integer (nullable = true)
 |-- ISBN: string (nullable = true)
 |-- Book-Rating: integer (nullable = true)
 |-- Book-Title: string (nullable = true)
 |-- Book-Author: string (nullable = true)
 |-- Year-Of-Publication: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Age: double (nullable = true)

## **_Data preprocessing and Descriptive statistics_**

In [13]:
from pyspark.sql import functions as F
rating_counts = abook.groupBy('Book-Rating').count().sort('Book-Rating')
rating_counts.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------+
|Book-Rating| count|
+-----------+------+
|          0|647294|
|          1|  1481|
|          2|  2375|
|          3|  5118|
|          4|  7617|
|          5| 45355|
|          6| 31687|
|          7| 66402|
|          8| 91804|
|          9| 60778|
|         10| 71225|
+-----------+------+

In [14]:
sc.install_pypi_package("pandas")  #sc.install_pypi_package("pandas==0.25.1")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
sc.install_pypi_package("matplotlib")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
sc.install_pypi_package("numpy")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
sc.install_pypi_package("seaborn")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
# summary statistics
summary = abook.describe()
summary.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+--------------------+-----------------+--------------------+--------------+-------------------+--------------------+--------------------+-----------------+
|summary|          User-ID|                ISBN|      Book-Rating|          Book-Title|   Book-Author|Year-Of-Publication|           Publisher|            Location|              Age|
+-------+-----------------+--------------------+-----------------+--------------------+--------------+-------------------+--------------------+--------------------+-----------------+
|  count|          1031136|             1031136|          1031136|             1031136|       1031135|            1031136|             1031134|             1031136|           753301|
|   mean|140594.5035058421| 7.713305080609732E8|2.839051298761754|            Infinity|       2000.75| 1968.1949149090515|  25032.333333333332|                null|37.39764848314286|
| stddev| 80524.6590394707|1.1111196740559175E9|3.854156635175236|                 Na

In [19]:
##### null value count #####

from pyspark.sql.functions import col
null_counts = [abook.filter(col(c).isNull()).count() for c in abook.columns]
null_counts_abook = spark.createDataFrame(zip(abook.columns, null_counts), ['Column', 'Null_Count'])
null_counts_abook.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------+
|             Column|Null_Count|
+-------------------+----------+
|            User-ID|         0|
|               ISBN|         0|
|        Book-Rating|         0|
|         Book-Title|         0|
|        Book-Author|         1|
|Year-Of-Publication|         0|
|          Publisher|         2|
|           Location|         0|
|                Age|    277835|
+-------------------+----------+

In [20]:
####shape of the DataFrame #######

num_rows = abook.count()
num_cols = len(abook.columns)
print("Shape of the DataFrame:", (num_rows, num_cols))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Shape of the DataFrame: (1031136, 9)

In [21]:
############ Unique count ##################

unique_user_count = abook.select("User-ID").distinct().count()
print("Number of unique User:", unique_user_count)
unique_ISBN_count = abook.select('ISBN').distinct().count()
print("Number of unique ISBN:", unique_ISBN_count)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of unique User: 92106
Number of unique ISBN: 270151

In [22]:
#abook = abook.select(abook["User-ID"].cast("integer"),
                         #abook["ISBN"].cast("integer"),
                       #abook["Book-Rating"].cast("float"))

#abook.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
from pyspark.sql.functions import regexp_replace

abook = abook.withColumn("Year-Of-Publication", regexp_replace("Year-Of-Publication", "[^0-9]", ""))
abook = abook.withColumn("Year-Of-Publication", col("Year-Of-Publication").cast("integer"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
abook = abook.na.drop(subset=['User-ID', 'ISBN', 'Book-Rating','Year-Of-Publication','Book-Author','Publisher','Age'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
aa= abook.describe()
aa.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+--------------------+-----------------+--------------------+--------------+-------------------+--------------------+--------------------+------------------+
|summary|           User-ID|                ISBN|      Book-Rating|          Book-Title|   Book-Author|Year-Of-Publication|           Publisher|            Location|               Age|
+-------+------------------+--------------------+-----------------+--------------------+--------------+-------------------+--------------------+--------------------+------------------+
|  count|            753296|              753296|           753296|              753296|        753296|             753296|              753296|              753296|            753296|
|   mean|141231.20758108367| 7.943365196115569E8|2.768895626685924|            Infinity|          null| 1967.6707243898813|             18775.0|                null| 37.39765776003058|
| stddev| 78828.59480816725|1.1623927382564003E9|3.864231387680776|        

In [26]:
year = abook.select("Year-Of-Publication").distinct()
year.sort("Year-Of-Publication", ascending = True).show()
year.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|Year-Of-Publication|
+-------------------+
|                  0|
|               1376|
|               1378|
|               1897|
|               1900|
|               1901|
|               1902|
|               1908|
|               1909|
|               1911|
|               1914|
|               1917|
|               1920|
|               1921|
|               1922|
|               1923|
|               1924|
|               1925|
|               1926|
|               1927|
+-------------------+
only showing top 20 rows

109

In [27]:
#### check after drop NA
from pyspark.sql.functions import col
null_counts = [abook.filter(col(c).isNull()).count() for c in abook.columns]
null_counts_abook = spark.createDataFrame(zip(abook.columns, null_counts), ['Column', 'Null_Count'])
null_counts_abook.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------+
|             Column|Null_Count|
+-------------------+----------+
|            User-ID|         0|
|               ISBN|         0|
|        Book-Rating|         0|
|         Book-Title|         0|
|        Book-Author|         0|
|Year-Of-Publication|         0|
|          Publisher|         0|
|           Location|         0|
|                Age|         0|
+-------------------+----------+

In [28]:
from pyspark.sql.functions import avg, count

# average rating for each user
average_ratings = abook.groupBy("User-ID").agg(avg("Book-Rating").alias("average_rating"), count("Book-Rating").alias("rating_count"))
average_ratings.sort("average_rating", ascending = True).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------+------------+
|User-ID|average_rating|rating_count|
+-------+--------------+------------+
| 210010|           0.0|          24|
| 143820|           0.0|           1|
| 278235|           0.0|           1|
|  80155|           0.0|           1|
|  65787|           0.0|           1|
| 232380|           0.0|           3|
|  31172|           0.0|          27|
| 156421|           0.0|           5|
| 255320|           0.0|           2|
| 214098|           0.0|           1|
| 208551|           0.0|           5|
|  18217|           0.0|           1|
| 204247|           0.0|           1|
|  30857|           0.0|           2|
| 173830|           0.0|           2|
|  62882|           0.0|           1|
| 233853|           0.0|          27|
| 231911|           0.0|           1|
| 264678|           0.0|           2|
| 276184|           0.0|           8|
+-------+--------------+------------+
only showing top 20 rows

In [29]:
#Filter out users with rating_count > 5 and average_rating == 0
#Ensures that we retain users who have provided some ratings but removes outliers who may have a large number of ratings with an average of 0, 
filtered_users = average_ratings.filter(~((average_ratings.rating_count > 5) & (average_ratings.average_rating == 0)))
filtered_users = filtered_users.filter("rating_count <= 50")
filtered_users = filtered_users.withColumnRenamed("User-ID", "filtered_User-ID")
filtered_users.sort("rating_count", ascending = False).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------------+------------+
|filtered_User-ID|average_rating|rating_count|
+----------------+--------------+------------+
|          137437|          1.84|          50|
|          154844|          5.86|          50|
|          134784|          2.24|          50|
|           91058|          4.86|          50|
|          270828|          2.12|          50|
|           51184|          6.02|          50|
|           71981|           2.7|          50|
|          190939|          1.46|          50|
|          131482|          0.74|          50|
|            5483|          0.16|          50|
|           41701|          2.24|          50|
|          123988|           4.2|          50|
|          187598|          7.18|          50|
|          119594|          3.84|          50|
|          261685|          1.36|          50|
|           97721|           3.5|          50|
|            4785|          4.16|          50|
|          231263|          4.18|          50|
|            

In [30]:
joined_data = abook.join(filtered_users, abook['User-ID'] == filtered_users['filtered_User-ID'], "inner")
# Drop the duplicated User-ID column
joined_data = joined_data.drop("filtered_User-ID")
joined_data.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+-----------+--------------------+--------------------+-------------------+--------------------+--------------------+----+------------------+------------+
|User-ID|      ISBN|Book-Rating|          Book-Title|         Book-Author|Year-Of-Publication|           Publisher|            Location| Age|    average_rating|rating_count|
+-------+----------+-----------+--------------------+--------------------+-------------------+--------------------+--------------------+----+------------------+------------+
| 184483|0312243022|          0| The Hours : A Novel|  Michael Cunningham|               2000|             Picador|pittsburgh, penns...|41.0|0.7435897435897436|          39|
| 184483|0375709045|          0|She's Gone Countr...|        Kyle Spencer|               2002|   Vintage Books USA|pittsburgh, penns...|41.0|0.7435897435897436|          39|
| 184483|0451167317|          0|       The Dark Half|        Stephen King|               1994|         Signet Book|pittsburgh, pen

In [31]:
joined_data.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

219597

In [32]:
users = joined_data.select("User-ID").distinct()
users.show()
users.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+
|User-ID|
+-------+
| 184483|
|  78440|
|  15846|
| 108806|
| 214084|
| 236323|
| 182760|
|  14423|
| 165781|
| 277710|
| 149589|
| 152023|
| 227671|
| 197609|
| 129410|
| 171406|
| 223066|
|  16999|
| 117278|
|  26490|
+-------+
only showing top 20 rows

51145

In [33]:
from pyspark.sql.functions import col
null_counts2 = [joined_data.filter(col(c).isNull()).count() for c in joined_data.columns]
null_counts_abook = spark.createDataFrame(zip(joined_data.columns, null_counts2), ['Column', 'Null_Count'])
null_counts_abook.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------+
|             Column|Null_Count|
+-------------------+----------+
|            User-ID|         0|
|               ISBN|         0|
|        Book-Rating|         0|
|         Book-Title|         0|
|        Book-Author|         0|
|Year-Of-Publication|         0|
|          Publisher|         0|
|           Location|         0|
|                Age|         0|
|     average_rating|         0|
|       rating_count|         0|
+-------------------+----------+

In [34]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, when
from pyspark.sql.functions import split, explode, col, count, collect_list, when

# Define age ranges
age_ranges = [(0, 18), (19, 29), (30, 39), (40, 49), (50, 59), (60, 69), (70, float('inf'))]  # Example age ranges

# Create a when-otherwise statement to map age to its corresponding range
age_range_column = when(
    (col("Age") >= 0) & (col("Age") <= 18), "0-18"
).when(
    (col("Age") >= 19) & (col("Age") <= 29), "19-29"
).when(
    (col("Age") >= 30) & (col("Age") <= 39), "30-39"
).when(
    (col("Age") >= 40) & (col("Age") <= 49), "40-49"
).when(
    (col("Age") >= 50) & (col("Age") <= 59), "50-59"
).when(
    (col("Age") >= 60) & (col("Age") <= 69), "60-69"
).otherwise("70+").alias("AgeRange")

# Step 2: Split the Book-Author string to extract individual authors
split_authors_data = joined_data.select(age_range_column, explode(split("Book-Author", ",")).alias("Author"))

# Step 3: Aggregate data based on user age range and author
author_counts = split_authors_data.groupBy("AgeRange", "Author").agg(count("*").alias("count"))

# Step 4: Identify the most popular authors for each age range
window_spec = Window.partitionBy("AgeRange").orderBy(col("count").desc())
popular_authors = author_counts.withColumn("rank", row_number().over(window_spec)) \
    .filter(col("rank") <= 5) \
    .groupBy("AgeRange").agg(collect_list("Author").alias("Popular Authors"))

# Show the results
popular_authors.sort("AgeRange", ascending=True).show(truncate=False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------------------------------------------------------------------------------+
|AgeRange|Popular Authors                                                                |
+--------+-------------------------------------------------------------------------------+
|0-18    |[R. L. Stine, J. K. Rowling, Stephen King, Rich Shapero, Ann M. Martin]        |
|19-29   |[Stephen King, Rich Shapero, John Grisham, Anne Rice, J. K. Rowling]           |
|30-39   |[Stephen King, John Grisham, Nora Roberts, James Patterson, Dean R. Koontz]    |
|40-49   |[Stephen King, John Grisham, Nora Roberts, James Patterson, Tom Clancy]        |
|50-59   |[James Patterson, John Grisham, Nora Roberts, Stephen King, Tom Clancy]        |
|60-69   |[Nora Roberts, James Patterson, John Grisham, Stephen King, Mary Higgins Clark]|
|70+     |[John Grisham, Nora Roberts, James Patterson, Rich Shapero, Robert Ludlum]     |
+--------+-------------------------------------------------------------------------------+

## _Most popular authors and books for different age groups_

In [35]:
from pyspark.sql.functions import split, explode, col, count, collect_list, when, avg, expr

# Age ranges definition
age_ranges = [(0, 18), (19, 29), (30, 39), (40, 49), (50, 59), (60, 69), (70, float('inf'))]  
age_range_column = when(
    (col("Age") >= 0) & (col("Age") <= 18), "0-18"
).when(
    (col("Age") >= 19) & (col("Age") <= 29), "19-29"
).when(
    (col("Age") >= 30) & (col("Age") <= 39), "30-39"
).when(
    (col("Age") >= 40) & (col("Age") <= 49), "40-49"
).when(
    (col("Age") >= 50) & (col("Age") <= 59), "50-59"
).when(
    (col("Age") >= 60) & (col("Age") <= 69), "60-69"
).otherwise("70+").alias("AgeRange")

# Split the Book-Author string to extract individual authors
split_authors_data = joined_data.select(age_range_column, explode(split("Book-Author", ",")).alias("Author"), "Book-Rating")

# Aggregate data based on user age range and author, considering ratings
author_rating_counts = split_authors_data.groupBy("AgeRange", "Author").agg(count("*").alias("Count"), avg("Book-Rating").alias("AverageRating"))

# Identify the most popular authors for each age range based on both count and average rating
popular_authors = author_rating_counts.orderBy(col("Count").desc(), col("AverageRating").desc()) \
                    .groupBy("AgeRange").agg(collect_list("Author").alias("Popular Authors"))

# Show top 5 popular authors for each age range
popular_authors = popular_authors.withColumn("Popular Authors", expr("slice(`Popular Authors`, 1, 5)"))

popular_authors.sort("AgeRange", ascending=True).show(truncate=False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+------------------------------------------------------------------------------------------+
|AgeRange|Popular Authors                                                                           |
+--------+------------------------------------------------------------------------------------------+
|0-18    |[Lars Saabye Christensen, Lindsay Caldwell, Mary Packard, Robyn Sisman, Hayao Miyazaki]   |
|19-29   |[Aphra Behn, Deb Kastner, Juan Marse, Wordsworth, Julie Aigner-Clark]                     |
|30-39   |[ Dr. Hessayon, Pang-Mei Natasha Chang, Robert Arthur, NATALIE GOLDBERG, Bernard Lenteric]|
|40-49   |[Diane Baker Mason, Lisa Goldstein, Eve Bunting, Angelica Rose, Tom Topor]                |
|50-59   |[Francine Mathews, Carrie Fisher, Kay R. Jamison, Seamus Heaney, Nancy Atherton]          |
|60-69   |[Aaron Elkins, Mark Haddon, Phil Rickman, Rachel Naomi Remen, Steve Ettlinger]            |
|70+     |[Janet Dailey, Arturo Perez-Reverte, Matthew Kneale, NORA ROBERTS, Katha

In [36]:
# Do the same steps as popular Authors, but change to be 'Book title'

split_title_data = joined_data.select(age_range_column, col("Book-Title").alias("Title"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
title_counts = split_title_data.groupBy("AgeRange", "Title").agg(count("*").alias("count"))

window_spec = Window.partitionBy("AgeRange").orderBy(col("count").desc())
popular_title = title_counts.withColumn("rank", row_number().over(window_spec)) \
    .filter(col("rank") <= 5) \
    .groupBy("AgeRange").agg(collect_list("Title").alias("Popular Books"))

popular_title.sort("AgeRange", ascending=True).show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|AgeRange|Popular Books                                                                                                                                                                               |
+--------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0-18    |[Wild Animus, And Then There Were None : A Novel, Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback)), To Kill a Mockingbird, Harry Potter and the Chamber of Secrets (Book 2)]|
|19-29   |[Wild Animus, The Lovely Bones: A Novel, Bridget Jones's Diary, The Da Vinci Code, Interview with the Vampire]                                                                              |


## **_ALS model: Top 5 recommendations for each user_**

In [38]:
# Filter out rows with null values in the 'ISBN' column
#abook_filtered = final_data_clean.filter(abook["ISBN"].isNotNull())

# Drop any remaining null values in other specified columns
#abook_filtered = abook_filtered.na.drop(subset=["User-ID", "Book-Rating"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer

isbn_column = joined_data.select("ISBN")
indexer = StringIndexer(inputCol="ISBN", outputCol="ISBN_Index")
indexer_model = indexer.fit(isbn_column)
df_indexed = indexer_model.transform(joined_data)
df_indexed.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+-----------+--------------------+------------------+-------------------+--------------------+--------------------+----+------------------+------------+----------+
|User-ID|      ISBN|Book-Rating|          Book-Title|       Book-Author|Year-Of-Publication|           Publisher|            Location| Age|    average_rating|rating_count|ISBN_Index|
+-------+----------+-----------+--------------------+------------------+-------------------+--------------------+--------------------+----+------------------+------------+----------+
|  78440|0345285980|          9|      Crystal singer|    Anne McCaffrey|               1982|          Ballantine| houston, texas, usa|52.0| 6.368421052631579|          19|    4946.0|
| 250301|0192833170|          7|Miss Julie and Ot...| August Strindberg|               1999|Oxford University...|new york, new yor...|21.0| 8.666666666666666|           9|   35754.0|
|  12626|0330314971|          0|   Focaults Pendulum|       Umberto Eco|             

In [40]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import when, col

(training, test) = df_indexed.randomSplit([0.8, 0.2])

als = ALS(rank=10, maxIter=10, regParam=0.01, userCol="User-ID", itemCol="ISBN_Index", ratingCol="Book-Rating",
          coldStartStrategy="drop",nonnegative=True,implicitPrefs=True)
model = als.fit(training)
####################


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
predictions = model.transform(test)
predictions.sort("User-ID", ascending = True).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+-----------+--------------------+--------------------+-------------------+--------------------+--------------------+----+-----------------+------------+----------+-------------+
|User-ID|      ISBN|Book-Rating|          Book-Title|         Book-Author|Year-Of-Publication|           Publisher|            Location| Age|   average_rating|rating_count|ISBN_Index|   prediction|
+-------+----------+-----------+--------------------+--------------------+-------------------+--------------------+--------------------+----+-----------------+------------+----------+-------------+
|      2|0195153448|          0| Classical Mythology|  Mark P. O. Morford|               2002|Oxford University...|stockton, califor...|18.0|              0.0|           1|   35973.0|          0.0|
|     36|067176537X|          0|The Therapeutic T...|     Dolores Krieger|               1979|            Fireside|montreal, quebec,...|24.0|              0.0|           1|   23614.0|          0.0|
|     44|0

In [42]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="Book-Rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) = " + str(rmse))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) = 5.715631039690308

In [43]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

als = ALS(userCol="User-ID", itemCol="ISBN_Index", ratingCol="Book-Rating", coldStartStrategy="drop",implicitPrefs=True)

# Define the parameter grid
param_grid = ParamGridBuilder() \
    .addGrid(als.rank, [10, 15]) \
    .addGrid(als.maxIter, [10, 20]) \
    .addGrid(als.regParam, [0.01, 0.2]) \
    .build()

evaluator = RegressionEvaluator(metricName="rmse", labelCol="Book-Rating", predictionCol="prediction")
cross_validator = CrossValidator(estimator=als,
                                 estimatorParamMaps=param_grid,
                                 evaluator=evaluator,
                                 numFolds=5)
cv_model = cross_validator.fit(df_indexed)
best_model = cv_model.bestModel

print("Best Model Parameters:")
print("Rank:", best_model.rank)
print("Max Iterations:", best_model._java_obj.parent().getMaxIter())
print("Regularization Parameter:", best_model._java_obj.parent().getRegParam())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Best Model Parameters:
Rank: 15
Max Iterations: 20
Regularization Parameter: 0.01

In [44]:
als = ALS(rank=15, maxIter=20, regParam=0.2, userCol="User-ID", itemCol="ISBN_Index", ratingCol="Book-Rating",
          coldStartStrategy="drop",nonnegative=True,implicitPrefs=True)
model = als.fit(training)
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="Book-Rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) = " + str(rmse))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Root Mean Squared Error (RMSE) = 5.714561412390321

In [45]:
# top 5 books recommendations for each user
user_recs = model.recommendForAllUsers(5)
user_recs.show(truncate=False)
user_recs.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------------------------------------------------------------------------------------------+
|User-ID|recommendations                                                                                       |
+-------+------------------------------------------------------------------------------------------------------+
|133    |[{1, 0.04258638}, {2, 0.0390005}, {11, 0.030625638}, {47, 0.02584606}, {18, 0.02559783}]              |
|406    |[{44905, 0.0}, {44895, 0.0}, {44865, 0.0}, {44855, 0.0}, {44854, 0.0}]                                |
|444    |[{44905, 0.0}, {44895, 0.0}, {44865, 0.0}, {44855, 0.0}, {44854, 0.0}]                                |
|496    |[{21, 0.30984187}, {71, 0.2480533}, {79, 0.23126993}, {73, 0.20138165}, {102, 0.19699991}]            |
|685    |[{44905, 0.0}, {44895, 0.0}, {44865, 0.0}, {44855, 0.0}, {44854, 0.0}]                                |
|744    |[{44905, 0.0}, {44895, 0.0}, {44865, 0.0}, {44855, 0.0}, {44854, 0.0}]                 

In [46]:
# Look at the books recommended to user 512
print("User 444's Recommendations:")
user_no = user_recs.filter(col("User-ID") == 512)
user_no.show(truncate=False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

User 512's Recommendations:
+-------+-----------------------------------------------------------------------------------------------+
|User-ID|recommendations                                                                                |
+-------+-----------------------------------------------------------------------------------------------+
|512    |[{21, 0.029961674}, {71, 0.024139864}, {79, 0.02251021}, {73, 0.019866863}, {102, 0.019422736}]|
+-------+-----------------------------------------------------------------------------------------------+

In [47]:
extract_title = df_indexed.dropDuplicates(["ISBN_Index"])
extract_title.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+-----------+--------------------+--------------------+-------------------+--------------------+--------------------+----+------------------+------------+----------+
|User-ID|      ISBN|Book-Rating|          Book-Title|         Book-Author|Year-Of-Publication|           Publisher|            Location| Age|    average_rating|rating_count|ISBN_Index|
+-------+----------+-----------+--------------------+--------------------+-------------------+--------------------+--------------------+----+------------------+------------+----------+
|   5037|0440226430|          6|      Summer Sisters|          Judy Blume|               1999|Dell Publishing C...|dodge center, min...|26.0| 4.791666666666667|          24|      29.0|
| 275306|0446310786|         10|To Kill a Mocking...|          Harper Lee|               1988|Little Brown &amp...|   tyler, texas, usa|24.0| 2.608695652173913|          46|      47.0|
| 197424|067102423X|          5|        Bag of Bones|        Stephen King| 

In [48]:
from pyspark.sql.functions import col, explode

# Explode the recommendations column to get each recommendation as a separate row
exploded_recs = user_recs.select("User-ID", explode("recommendations").alias("rec"))

# Extract ISBN indices and ratings from the exploded DataFrame
top_5_indices_and_ratings = exploded_recs.filter(col("User-ID") == 512) \
                                        .select(col("rec.ISBN_Index"), col("rec.rating")) \
                                        .orderBy(col("rec.rating").desc()) \
                                        .limit(5)

# Join with book_titles DataFrame to get book titles
user_top_5_titles = top_5_indices_and_ratings.join(extract_title, top_5_indices_and_ratings.ISBN_Index == df_indexed.ISBN_Index, how='inner') \
                                              .select("Book-Title",'User-ID',df_indexed['ISBN_Index'])

# Show top 5 recommended book titles for User-ID 512
user_top_5_titles.show(truncate=False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+-------+----------+
|Book-Title                    |User-ID|ISBN_Index|
+------------------------------+-------+----------+
|Snow Falling on Cedars        |100815 |21.0      |
|Fahrenheit 451                |82122  |71.0      |
|The Beach House               |87960  |79.0      |
|Fall On Your Knees (Oprah #45)|109084 |102.0     |
|Mystic River                  |84900  |73.0      |
+------------------------------+-------+----------+

In [49]:
from pyspark.sql.functions import col

# Look at the books recommended to user 85
print("User 512's Recommendations:")
user_no = user_recs.filter(col("User-ID") == 512)
user_no.show(truncate=False)

# Extract recommendation ratings for User 
ratings_for_user_85 = exploded_recs.filter(col("User-ID") == 512) \
                                   .select("rec.ISBN_Index", "rec.rating")

# Join with book_titles DataFrame to get book titles
user_85_recommendations_with_ratings = ratings_for_user_85.join(extract_title, ratings_for_user_85["ISBN_Index"] == df_indexed["ISBN_Index"], how='inner') \
                                                          .select("Book-Title", ratings_for_user_85["rating"].alias("Recommendation Rating"))

print("User 512's Recommendations with Ratings:")
user_85_recommendations_with_ratings.show(truncate=False)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

User 512's Recommendations:
+-------+-----------------------------------------------------------------------------------------------+
|User-ID|recommendations                                                                                |
+-------+-----------------------------------------------------------------------------------------------+
|512    |[{21, 0.029961674}, {71, 0.024139864}, {79, 0.02251021}, {73, 0.019866863}, {102, 0.019422736}]|
+-------+-----------------------------------------------------------------------------------------------+

User 512's Recommendations with Ratings:
+------------------------------+---------------------+
|Book-Title                    |Recommendation Rating|
+------------------------------+---------------------+
|Snow Falling on Cedars        |0.029961674          |
|The Beach House               |0.02251021           |
|Fahrenheit 451                |0.024139864          |
|Fall On Your Knees (Oprah #45)|0.019422736          |
|Mystic River  

In [50]:
top_5_indices_and_ratings.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------+
|ISBN_Index|     rating|
+----------+-----------+
|        21|0.029961674|
|        71|0.024139864|
|        79| 0.02251021|
|        73|0.019866863|
|       102|0.019422736|
+----------+-----------+

In [51]:
################################################################
users_check = df_indexed.select("User-ID").distinct()
users_check.sort("User-ID", ascending = True).show()
users_check.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+
|User-ID|
+-------+
|      2|
|     10|
|     19|
|     20|
|     36|
|     42|
|     44|
|     51|
|     56|
|     64|
|     67|
|     70|
|     75|
|     78|
|     79|
|     85|
|     92|
|     99|
|    109|
|    114|
+-------+
only showing top 20 rows

51145

In [52]:
# generate top 5 books recommendations for a specified set of users
users =  df_indexed.select(als.getUserCol()).distinct().limit(10)
users.show()
user_subset_recs = model.recommendForUserSubset(users, 5)
user_subset_recs.show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+
|User-ID|
+-------+
| 275481|
| 184483|
|    496|
|  22097|
|  19206|
|  26801|
| 250301|
| 191536|
| 101971|
|  73436|
+-------+

+-------+------------------------------------------------------------------------------------------------+
|User-ID|recommendations                                                                                 |
+-------+------------------------------------------------------------------------------------------------+
|275481 |[{14, 0.035990104}, {22, 0.034616664}, {26, 0.03401068}, {32, 0.032490447}, {25, 0.03142964}]   |
|26801  |[{31, 0.12127166}, {88, 0.12112089}, {80, 0.10558838}, {215, 0.10220046}, {84, 0.101102754}]    |
|101971 |[{11, 0.04695988}, {18, 0.040131275}, {23, 0.03898678}, {199, 0.031032933}, {56, 0.030989358}]  |
|250301 |[{22, 0.3399341}, {59, 0.27037272}, {118, 0.26079166}, {92, 0.24471638}, {105, 0.22854473}]     |
|184483 |[{8, 0.5169407}, {3, 0.5065312}, {5, 0.48664728}, {6, 0.48600617}, {13, 0.43421867}]            |
|22

In [53]:
user_subset_recs.take(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(User-ID=113630, recommendations=[Row(ISBN_Index=7, rating=0.4491007924079895), Row(ISBN_Index=36, rating=0.3713008165359497), Row(ISBN_Index=99, rating=0.37008288502693176), Row(ISBN_Index=66, rating=0.31736451387405396), Row(ISBN_Index=146, rating=0.26530057191848755)])]

In [54]:
import pyspark.sql.functions as fn

user_books = user_subset_recs.select('User-ID', fn.explode('recommendations').alias('Book_rating'))
user_books.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------------+
|User-ID|     Book_rating|
+-------+----------------+
| 146081|{32, 0.23502888}|
| 146081|{25, 0.23445165}|
| 146081|{39, 0.21676286}|
| 146081|{14, 0.20983696}|
| 146081|{33, 0.20780882}|
| 161751|    {44905, 0.0}|
| 161751|    {44895, 0.0}|
| 161751|    {44865, 0.0}|
| 161751|    {44855, 0.0}|
| 161751|    {44854, 0.0}|
| 101835|    {44905, 0.0}|
| 101835|    {44895, 0.0}|
| 101835|    {44865, 0.0}|
| 101835|    {44855, 0.0}|
| 101835|    {44854, 0.0}|
|  51627|  {8, 0.4352503}|
|  51627| {3, 0.42443034}|
|  51627| {6, 0.40750784}|
|  51627|{13, 0.37428597}|
|  51627|{38, 0.33631706}|
+-------+----------------+
only showing top 20 rows

In [55]:
#spark.stop()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…